In [3]:
#Install necessary libraries

import numpy as np
import pandas as pd
import requests
# KO : pip install folium
# KO : conda install -c conda-forge folium
# KO : python3 -m pip install folium
!pip install folium
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
from sklearn.cluster import KMeans

#import geopy
#from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize

# add bs4 library:
from bs4 import BeautifulSoup
import lxml

import types

print('Libraries imported.')

Libraries imported.


In [4]:
myCP=pd.read_csv('244400404_base-codes-postaux-nantes-metropole.csv',delimiter=';')
myCP.head()

,Code_INSEE,Nom_commune,Code_postal,latitude,longitude
0,44009,BASSE GOULAINE,44115,47.212014,-1.462469
1,44143,REZE,44400,47.176234,-1.549664
2,44150,ST AIGNAN GRANDLIEU,44860,47.133107,-1.623171
3,44171,ST LEGER LES VIGNES,44710,47.142969,-1.714036
4,44194,SAUTRON,44880,47.273227,-1.680473


In [21]:
# create the map of Nantes with the postal codes :
import folium
map_nantes = folium.Map(location=[47.2173, -1.5534], zoom_start=12)
for lat, lng, Code_INSEE, Nom_commune in zip(
        myCP['latitude'], 
        myCP['longitude'], 
        myCP['Code_INSEE'], 
        myCP['Nom_commune']):
    label = '{}, {}'.format(Code_INSEE, Nom_commune)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='#000000',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nantes)  

map_nantes


In [5]:
my_equip_44=pd.read_csv('234400034_004-009_activites-des-fiches-equipements-rpdl_CL_44.csv',delimiter=';')
my_equip_44_foot=pd.read_csv('234400034_004-009_activites-des-fiches-equipements-rpdl_CL_44.csv',delimiter=';')
my_equip_44_basket=pd.read_csv('234400034_004-009_activites-des-fiches-equipements-rpdl_CL_44.csv',delimiter=';')
my_equip_44_swim=pd.read_csv('234400034_004-009_activites-des-fiches-equipements-rpdl_CL_44.csv',delimiter=';')
del my_equip_44_foot['Departement']
del my_equip_44_foot['Numero_fiche_equipement']
del my_equip_44_foot['code_region']
del my_equip_44_foot['Libelle_departement']
del my_equip_44_foot['Activite_praticable']
del my_equip_44_foot['Activite_pratiquee']
del my_equip_44_foot['Salle_specialisable']
del my_equip_44_foot['Activite code']
del my_equip_44_basket['Departement']
del my_equip_44_basket['Numero_fiche_equipement']
del my_equip_44_basket['code_region']
del my_equip_44_basket['Libelle_departement']
del my_equip_44_basket['Activite_praticable']
del my_equip_44_basket['Activite_pratiquee']
del my_equip_44_basket['Salle_specialisable']
del my_equip_44_basket['Activite code']
del my_equip_44_swim['Departement']
del my_equip_44_swim['Numero_fiche_equipement']
del my_equip_44_swim['code_region']
del my_equip_44_swim['Libelle_departement']
del my_equip_44_swim['Activite_praticable']
del my_equip_44_swim['Activite_pratiquee']
del my_equip_44_swim['Salle_specialisable']
del my_equip_44_swim['Activite code']
my_equip_44.head()

,Departement,Libelle_departement,Code_INSEE,Nom_commune,Numero_fiche_equipement,Nombre_equipements,Activite code,Activite_libelle,Activite_praticable,Activite_pratiquee,Salle_specialisable,Niveau_activite,lat,long,code_region
0,44,Loire-Atlantique,44219,Vritz,189437,1,2204,Vtt (Cross Country/ Descente/ Trial/ Rallye/ F...,Oui,Oui,Non,Loisir - Entretien - Remise en forme,47.589046,-1.072706,52
1,44,Loire-Atlantique,44220,Vue,73634,1,2901,Football / Football en salle (Futsal),Oui,Oui,Non,Competition departementale,47.194197,-1.900877,52
2,44,Loire-Atlantique,44221,Chevallerais,254428,1,3501,Gymnastique volontaire,Oui,Oui,Non,Loisir - Entretien - Remise en forme,47.477184,-1.664932,52
3,44,Loire-Atlantique,44222,Roche-Blanche,231660,1,3501,Gymnastique volontaire,Oui,Oui,Non,Loisir - Entretien - Remise en forme,47.437743,-1.144298,52
4,44,Loire-Atlantique,44222,Roche-Blanche,312031,1,6001,Petanque et jeu provencal,Oui,Oui,Non,Loisir - Entretien - Remise en forme,47.437743,-1.144298,52


In [6]:
#Foot equipments data input
my_equip_44_foot['foot_bool'] = np.where(my_equip_44_foot['Activite_libelle']=="Football / Football en salle (Futsal)", 'oui', 'non')
# Get names of indexes for which column foot_bool has value non
indexNames = my_equip_44_foot[ my_equip_44_foot['foot_bool'] == 'non' ].index
# Delete these row indexes from dataFrame
my_equip_44_foot.drop(indexNames , inplace=True)
my_equip_44_foot.join (myCP.set_index( [ 'Code_INSEE' ], verify_integrity=True ), on=[ 'Code_INSEE' ], how='left' , lsuffix='_left', rsuffix='_right')
my_equip_44_foot.head()

,Code_INSEE,Nom_commune,Nombre_equipements,Activite_libelle,Niveau_activite,lat,long,foot_bool
1,44220,Vue,1,Football / Football en salle (Futsal),Competition departementale,47.194197,-1.900877,oui
7,44223,Geneston,1,Football / Football en salle (Futsal),Competition regionale,47.059661,-1.526029,oui
20,44109,Nantes,1,Football / Football en salle (Futsal),Loisir - Entretien - Remise en forme,47.231636,-1.548310,oui
29,44109,Nantes,1,Football / Football en salle (Futsal),Entrainement,47.231636,-1.548310,oui
77,44142,Remouille,1,Football / Football en salle (Futsal),Entrainement,47.038552,-1.387490,oui


In [58]:
# create the map of Nantes with the football equipments :
import folium
map_nantes_foot = folium.Map(location=[47.2173, -1.5534], zoom_start=10)
for lat, lng, Code_INSEE, Nom_commune in zip(
        my_equip_44_foot['lat'], 
        my_equip_44_foot['long'], 
        my_equip_44_foot['Code_INSEE'], 
        my_equip_44_foot['Nom_commune'] ):
    label = '{}, {}'.format(Code_INSEE, Nom_commune)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#008000',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nantes_foot)  

map_nantes_foot

In [7]:
#Basket equipments data input
my_equip_44_basket['basket_bool'] = np.where(my_equip_44_basket['Activite_libelle']=="Basket-Ball", 'oui', 'non')
# Get names of indexes for which column basket_bool has value non
indexNames = my_equip_44_basket[ my_equip_44_basket['basket_bool'] == 'non' ].index
# Delete these row indexes from dataFrame
my_equip_44_basket.drop(indexNames , inplace=True)
my_equip_44_basket.join (myCP.set_index( [ 'Code_INSEE' ], verify_integrity=True ), on=[ 'Code_INSEE' ], how='left' , lsuffix='_left', rsuffix='_right')
my_equip_44_basket.head()

,Code_INSEE,Nom_commune,Nombre_equipements,Activite_libelle,Niveau_activite,lat,long,basket_bool
17,44109,Nantes,1,Basket-Ball,Loisir - Entretien - Remise en forme,47.231636,-1.54831,oui
18,44109,Nantes,1,Basket-Ball,Loisir - Entretien - Remise en forme,47.231636,-1.54831,oui
19,44109,Nantes,1,Basket-Ball,Loisir - Entretien - Remise en forme,47.231636,-1.54831,oui
37,44109,Nantes,1,Basket-Ball,Loisir - Entretien - Remise en forme,47.231636,-1.54831,oui
38,44109,Nantes,1,Basket-Ball,Loisir - Entretien - Remise en forme,47.231636,-1.54831,oui


In [33]:
# create the map of Nantes with the basket equipments :
import folium
map_nantes_basket = folium.Map(location=[47.2173, -1.5534], zoom_start=10)
for lat, lng, Code_INSEE, Nom_commune in zip(
        my_equip_44_basket['lat'], 
        my_equip_44_basket['long'], 
        my_equip_44_basket['Code_INSEE'], 
        my_equip_44_basket['Nom_commune']):
    label = '{}, {}'.format(Code_INSEE, Nom_commune)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#FF0000',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nantes_basket)  

map_nantes_basket

In [8]:
#Swimming pools equipments data input
my_equip_44_swim['swim_bool'] = np.where(my_equip_44_swim['Activite_libelle']=="Natation course sportive", 'oui', 'non')
# Get names of indexes for which column swim_bool has value non
indexNames = my_equip_44_swim[ my_equip_44_swim['swim_bool'] == 'non' ].index
# Delete these row indexes from dataFrame
my_equip_44_swim.drop(indexNames , inplace=True)
my_equip_44_swim.join (myCP.set_index( [ 'Code_INSEE' ], verify_integrity=True ), on=[ 'Code_INSEE' ], how='left' , lsuffix='_left', rsuffix='_right')
my_equip_44_swim.head()


,Code_INSEE,Nom_commune,Nombre_equipements,Activite_libelle,Niveau_activite,lat,long,swim_bool
379,44084,Loroux-Bottereau,1,Natation course sportive,Entrainement,47.233398,-1.331702,oui
610,44015,Blain,1,Natation course sportive,Entrainement,47.462821,-1.767895,oui
688,44109,Nantes,1,Natation course sportive,Competition regionale,47.231636,-1.548310,oui
1218,44051,Derval,1,Natation course sportive,Entrainement,47.657680,-1.677730,oui
1386,44026,Carquefou,1,Natation course sportive,Competition departementale,47.301719,-1.472772,oui


In [86]:
# create the map of Nantes with the swim equipments :
import folium
map_nantes_swim = folium.Map(location=[47.2173, -1.5534], zoom_start=10)
for lat, lng, Code_INSEE, Nom_commune in zip(
        my_equip_44_swim['lat'], 
        my_equip_44_swim['long'], 
        my_equip_44_swim['Code_INSEE'], 
        my_equip_44_swim['Nom_commune']):
    label = '{}, {}'.format(Code_INSEE, Nom_commune)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#0000FF',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nantes_swim)  

map_nantes_swim

In [11]:
#Basical_statistics of equipments numbers by level of activities
#Foot
my_stat_level_foot=my_equip_44_foot[["Code_INSEE","Niveau_activite", "Nombre_equipements"]]
my_stat_level_foot2=my_equip_44_foot[["Code_INSEE","Niveau_activite", "Nombre_equipements"]]

#my_stat_level_foot.shape
# gives 855,3

my_stat_level_foot2["Nombre_equipements"].apply(pd.to_numeric)
my_stat_level_foot2=my_stat_level_foot.groupby(by=["Code_INSEE","Niveau_activite"]).sum().reset_index()

my_stat_level_foot2.loc[my_stat_level_foot2["Niveau_activite"] == "Competition departementale","Niveau_activite"] = "Competition"
my_stat_level_foot2.loc[my_stat_level_foot2["Niveau_activite"] == "Competition regionale","Niveau_activite"] = "Competition"
my_stat_level_foot2.loc[my_stat_level_foot2["Niveau_activite"] == "Competition nationale","Niveau_activite"] = "Competition"
my_stat_level_foot2.loc[my_stat_level_foot2["Niveau_activite"] == "Competition internationale","Niveau_activite"] = "Competition"
my_stat_level_foot2.loc[my_stat_level_foot2["Niveau_activite"] == "Entrainement","Niveau_activite"] = "Training"
my_stat_level_foot2.loc[my_stat_level_foot2["Niveau_activite"] == "Loisir - Entretien - Remise en forme","Niveau_activite"] = "Leisure"

my_stat_level_foot2

,Code_INSEE,Niveau_activite,Nombre_equipements
0,44001,Competition,2
1,44002,Competition,2
2,44002,Leisure,2
3,44003,Competition,2
4,44003,Competition,2
...,...,...,...
433,44222,Leisure,1
434,44223,Competition,2
435,44223,Leisure,1
436,44224,Competition,1


In [14]:
#Basical_statistics of equipments numbers by level of activities
#Basket
my_stat_level_basket=my_equip_44_basket[["Code_INSEE","Niveau_activite", "Nombre_equipements"]]
my_stat_level_basket2=my_equip_44_basket[["Code_INSEE","Niveau_activite", "Nombre_equipements"]]

# my_stat_level_basket.shape
my_stat_level_basket2["Nombre_equipements"].apply(pd.to_numeric)
my_stat_level_basket2=my_stat_level_basket.groupby(by=["Code_INSEE","Niveau_activite"]).sum().reset_index()

my_stat_level_basket2.loc[my_stat_level_basket2["Niveau_activite"] == "Competition departementale","Niveau_activite"] = "Competition"
my_stat_level_basket2.loc[my_stat_level_basket2["Niveau_activite"] == "Competition regionale","Niveau_activite"] = "Competition"
my_stat_level_basket2.loc[my_stat_level_basket2["Niveau_activite"] == "Competition nationale","Niveau_activite"] = "Competition"
my_stat_level_basket2.loc[my_stat_level_basket2["Niveau_activite"] == "Competition internationale","Niveau_activite"] = "Competition"
my_stat_level_basket2.loc[my_stat_level_basket2["Niveau_activite"] == "Entrainement","Niveau_activite"] = "Training"
my_stat_level_basket2.loc[my_stat_level_basket2["Niveau_activite"] == "Loisir - Entretien - Remise en forme","Niveau_activite"] = "Leisure"

my_stat_level_basket2


,Code_INSEE,Niveau_activite,Nombre_equipements
0,44001,Leisure,2
1,44002,Competition,1
2,44002,Competition,1
3,44002,Leisure,1
4,44003,Competition,1
...,...,...,...
327,44219,Leisure,1
328,44220,Leisure,1
329,44222,Leisure,1
330,44223,Competition,1


In [24]:
#Basical_statistics of equipments numbers by level of activities
#swim

my_stat_level_swim=my_equip_44_swim[["Code_INSEE","Niveau_activite", "Nombre_equipements"]]
my_stat_level_swim2=my_equip_44_swim[["Code_INSEE","Niveau_activite", "Nombre_equipements"]]
#my_stat_level_swim.shape


pd.to_numeric(my_stat_level_swim2.Nombre_equipements, downcast='integer')
my_stat_level_swim2=my_stat_level_swim2.groupby(by=["Code_INSEE","Niveau_activite"]).sum().reset_index()

#my_stat_level_swim2.shape


my_stat_level_swim2.loc[my_stat_level_swim2["Niveau_activite"] == "Competition departementale","Niveau_activite"] = "Competition"
my_stat_level_swim2.loc[my_stat_level_swim2["Niveau_activite"] == "Competition regionale","Niveau_activite"] = "Competition"
my_stat_level_swim2.loc[my_stat_level_swim2["Niveau_activite"] == "Competition nationale","Niveau_activite"] = "Competition"
my_stat_level_swim2.loc[my_stat_level_swim2["Niveau_activite"] == "Competition internationale","Niveau_activite"] = "Competition"
my_stat_level_swim2.loc[my_stat_level_swim2["Niveau_activite"] == "Entrainement","Niveau_activite"] = "Training"
my_stat_level_swim2.loc[my_stat_level_swim2["Niveau_activite"] == "Loisir - Entretien - Remise en forme","Niveau_activite"] = "Leisure"

my_stat_level_swim2

,Code_INSEE,Niveau_activite,Nombre_equipements
0,44003,Training,2
1,44012,Leisure,1
2,44015,Competition,1
3,44015,Training,1
4,44020,Training,1
5,44026,Competition,1
6,44036,Training,1
7,44036,Leisure,2
8,44043,Training,1
9,44051,Training,1


In [20]:
my_stat_level_foot3=my_stat_level_foot.groupby(by=["Nombre_equipements","Niveau_activite"]).agg('count')
my_stat_level_foot3.head()

Code_INSEE
Nombre_equipements Niveau_activite                       
1                  Competition departementale         376
                   Competition internationale           2
                   Competition nationale               11
                   Competition regionale               84
                   Entrainement                       150

In [21]:
my_stat_level_basket3=my_stat_level_basket.groupby(by=["Nombre_equipements","Niveau_activite"]).agg('count')
my_stat_level_basket3.head()

Code_INSEE
Nombre_equipements Niveau_activite                       
1                  Competition departementale         171
                   Competition internationale           1
                   Competition nationale               14
                   Competition regionale               85
                   Entrainement                        40

In [25]:
my_stat_level_swim3=my_stat_level_swim.groupby(by=["Nombre_equipements","Niveau_activite"]).agg('count')
my_stat_level_swim3.head()

Code_INSEE
Nombre_equipements Niveau_activite                                 
1                  Competition departementale                     6
                   Competition nationale                          3
                   Competition regionale                          4
                   Entrainement                                  25
                   Loisir - Entretien - Remise en forme           3

In [19]:
# FOOT : CLUSTERING PREPARATION
pre_clustering_44_foot = my_equip_44_foot[["Code_INSEE","lat","long","Niveau_activite",'Nombre_equipements']]
#pre_clustering_44_foot.shape
#(855,5)
pre_clustering_44_foot.loc[pre_clustering_44_foot["Niveau_activite"] == "Competition departementale","Niveau_activite"] = "foot_Competition"
pre_clustering_44_foot.loc[pre_clustering_44_foot["Niveau_activite"] == "Competition regionale","Niveau_activite"] = "foot_Competition"
pre_clustering_44_foot.loc[pre_clustering_44_foot["Niveau_activite"] == "Competition nationale","Niveau_activite"] = "foot_Competition"
pre_clustering_44_foot.loc[pre_clustering_44_foot["Niveau_activite"] == "Competition internationale","Niveau_activite"] = "foot_Competition"
pre_clustering_44_foot.loc[pre_clustering_44_foot["Niveau_activite"] == "Entrainement","Niveau_activite"] = "foot_Training"
pre_clustering_44_foot.loc[pre_clustering_44_foot["Niveau_activite"] == "Loisir - Entretien - Remise en forme","Niveau_activite"] = "foot_Leisure"
#729 rows × 5 columns

grouped_clustering_44_foot = pre_clustering_44_foot.groupby(["Code_INSEE","lat","long","Niveau_activite"])['Nombre_equipements'].sum().reset_index(name="sum_nb_equip")
grouped_clustering_44_foot.head()
#grouped_clustering_44_foot.shape
#(392,)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Code_INSEE,lat,long,Niveau_activite,sum_nb_equip
0,44001,47.556038,-1.493273,foot_Competition,2
1,44002,47.071926,-1.415565,foot_Competition,2
2,44002,47.071926,-1.415565,foot_Leisure,2
3,44003,47.383327,-1.182419,foot_Competition,4
4,44003,47.383327,-1.182419,foot_Leisure,1


In [20]:
# BASKET : CLUSTERING PREPARATION
pre_clustering_44_basket = my_equip_44_basket[["Code_INSEE","lat","long","Niveau_activite",'Nombre_equipements']]
# pre_clustering_44_basket=pre_clustering_44_basket.groupby(['Nombre_equipements']).min() : NO
pre_clustering_44_basket.loc[pre_clustering_44_basket["Niveau_activite"] == "Competition departementale","Niveau_activite"] = "basket_Competition"
pre_clustering_44_basket.loc[pre_clustering_44_basket["Niveau_activite"] == "Competition regionale","Niveau_activite"] = "basket_Competition"
pre_clustering_44_basket.loc[pre_clustering_44_basket["Niveau_activite"] == "Competition nationale","Niveau_activite"] = "basket_Competition"
pre_clustering_44_basket.loc[pre_clustering_44_basket["Niveau_activite"] == "Competition internationale","Niveau_activite"] = "basket_Competition"
pre_clustering_44_basket.loc[pre_clustering_44_basket["Niveau_activite"] == "Entrainement","Niveau_activite"] = "basket_Training"
pre_clustering_44_basket.loc[pre_clustering_44_basket["Niveau_activite"] == "Loisir - Entretien - Remise en forme","Niveau_activite"] = "basket_Leisure"
#2919 rows × 5 columns

grouped_clustering_44_basket = pre_clustering_44_basket.groupby(["Code_INSEE","lat","long","Niveau_activite"])['Nombre_equipements'].sum().reset_index(name="sum_nb_equip")
grouped_clustering_44_basket.head()
#grouped_clustering_44_basket.shape
#(609,)


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Code_INSEE,lat,long,Niveau_activite,sum_nb_equip
0,44001,47.556038,-1.493273,basket_Leisure,2
1,44002,47.071926,-1.415565,basket_Competition,2
2,44002,47.071926,-1.415565,basket_Leisure,1
3,44003,47.383327,-1.182419,basket_Competition,3
4,44003,47.383327,-1.182419,basket_Leisure,1


In [21]:
pre_clustering_44_swim = my_equip_44_swim[["Code_INSEE","lat","long","Niveau_activite",'Nombre_equipements']]
#del pre_clustering_44_swim['Nombre_equipements']
pre_clustering_44_swim.loc[pre_clustering_44_swim["Niveau_activite"] == "Competition departementale","Niveau_activite"] = "swim_Competition"
pre_clustering_44_swim.loc[pre_clustering_44_swim["Niveau_activite"] == "Competition regionale","Niveau_activite"] = "swim_Competition"
pre_clustering_44_swim.loc[pre_clustering_44_swim["Niveau_activite"] == "Competition nationale","Niveau_activite"] = "swim_Competition"
pre_clustering_44_swim.loc[pre_clustering_44_swim["Niveau_activite"] == "Competition internationale","Niveau_activite"] = "swim_Competition"
pre_clustering_44_swim.loc[pre_clustering_44_swim["Niveau_activite"] == "Entrainement","Niveau_activite"] = "swim_Training"
pre_clustering_44_swim.loc[pre_clustering_44_swim["Niveau_activite"] == "Loisir - Entretien - Remise en forme","Niveau_activite"] = "swim_Leisure"
#41 rows × 5 columns

grouped_clustering_44_swim = pre_clustering_44_swim.groupby(["Code_INSEE","lat","long","Niveau_activite"])['Nombre_equipements'].sum().reset_index(name="sum_nb_equip")
# no : grouped_clustering_44_swim.reset_index().rename(columns={'Nombre_equipements','SNE'})

grouped_clustering_44_swim.head()
#grouped_clustering_44_swim.shape
#(33,)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Code_INSEE,lat,long,Niveau_activite,sum_nb_equip
0,44003,47.383327,-1.182419,swim_Training,2
1,44012,47.085670,-2.031702,swim_Leisure,1
2,44015,47.462821,-1.767895,swim_Competition,1
3,44015,47.462821,-1.767895,swim_Training,1
4,44020,47.170906,-1.617398,swim_Training,1


In [22]:
# ALL SPORTS : CLUSTERING PREPARATION - REVIEWING COLUMNS
# pre_clustering_44_foot.columns
# GIVES : Index(['Code_INSEE', 'lat', 'long', 'Niveau_activite', 'Nombre_equipements'], dtype='object')
# pre_clustering_44_basket.columns
# GIVES : Index(['Code_INSEE', 'lat', 'long', 'Niveau_activite', 'Nombre_equipements'], dtype='object')
pre_clustering_44_swim.columns
# GIVES : Index(['Code_INSEE', 'lat', 'long', 'Niveau_activite', 'Nombre_equipements'], dtype='object')

Index(['Code_INSEE', 'lat', 'long', 'Niveau_activite', 'Nombre_equipements'], dtype='object')

In [23]:
# ALL SPORTS : CLUSTERING PREPARATION - JOINING DATAS

# NO grouped_clustering_44=grouped_clustering_44_foot["Code_INSEE","lat","long","Niveau_activite"]

#for join values convert values to string
#grouped_clustering_44_foot['Nombre_equipements'] = grouped_clustering_44_foot['Nombre_equipements'].astype(str)
#create mapping dict by dict comprehension
#L = ['1','2']
#d = {x: ','.join(L) for x in L}
#print (d)
#{'2': '1,2', '1': '1,2'}

#replace values by dict
#grouped_clustering_44_foot_2 = grouped_clustering_44_foot['SUB_NUM'].replace(d)



#grouped_clustering_44.join (grouped_clustering_44_basket.set_index( [ 'Code_INSEE',"lat","long","Niveau_activite" ] ), on=[ 'Code_INSEE',"lat","long","Niveau_activite" ], how='left' , lsuffix='_left', rsuffix='_right')
# grouped_clustering_44.join (grouped_clustering_44_swim.set_index( [ 'Code_INSEE',"lat","long","Niveau_activite" ] ), on=[ 'Code_INSEE',"lat","long","Niveau_activite" ], how='left' , lsuffix='_left', rsuffix='_right')

frames = [grouped_clustering_44_foot, grouped_clustering_44_basket, grouped_clustering_44_swim]
grouped_clustering_44 = pd.concat(frames)
grouped_clustering_44


#pre_clustering_44.shape
#(1034, 4)    note = 1034 = 33+609+392 !


,Code_INSEE,lat,long,Niveau_activite,sum_nb_equip
0,44001,47.556038,-1.493273,foot_Competition,2
1,44002,47.071926,-1.415565,foot_Competition,2
2,44002,47.071926,-1.415565,foot_Leisure,2
3,44003,47.383327,-1.182419,foot_Competition,4
4,44003,47.383327,-1.182419,foot_Leisure,1
...,...,...,...,...,...
28,44186,47.083994,-1.818145,swim_Training,1
29,44188,47.060291,-1.657114,swim_Training,1
30,44195,47.357065,-1.918421,swim_Training,1
31,44212,47.171515,-1.261466,swim_Training,1


In [24]:
grouped_clustering_44["Niveau_activite_NEW"] = grouped_clustering_44["Niveau_activite"] + '_' + grouped_clustering_44["sum_nb_equip"].apply(str)
del grouped_clustering_44["Niveau_activite"]
del grouped_clustering_44["sum_nb_equip"]
grouped_clustering_44

,Code_INSEE,lat,long,Niveau_activite_NEW
0,44001,47.556038,-1.493273,foot_Competition_2
1,44002,47.071926,-1.415565,foot_Competition_2
2,44002,47.071926,-1.415565,foot_Leisure_2
3,44003,47.383327,-1.182419,foot_Competition_4
4,44003,47.383327,-1.182419,foot_Leisure_1
...,...,...,...,...
28,44186,47.083994,-1.818145,swim_Training_1
29,44188,47.060291,-1.657114,swim_Training_1
30,44195,47.357065,-1.918421,swim_Training_1
31,44212,47.171515,-1.261466,swim_Training_1


In [25]:
# one hot encoding
clustering_44_onehot = pd.get_dummies(grouped_clustering_44[['Niveau_activite_NEW']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
clustering_44_onehot['Code_INSEE'] = grouped_clustering_44['Code_INSEE'] 

# move neighborhood column to the first column
fixed_columns = [clustering_44_onehot.columns[-1]] + list(clustering_44_onehot.columns[:-1])
clustering_44_onehot = clustering_44_onehot[fixed_columns]

clustering_44_onehot.head()

,Code_INSEE,basket_Competition_1,basket_Competition_10,basket_Competition_2,basket_Competition_3,basket_Competition_32,basket_Competition_4,basket_Competition_5,basket_Competition_7,basket_Competition_9,...,foot_Training_5,foot_Training_6,foot_Training_8,swim_Competition_1,swim_Competition_2,swim_Competition_5,swim_Leisure_1,swim_Leisure_2,swim_Training_1,swim_Training_2
0,44001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,44002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,44002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,44003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,44003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
clustering_44_onehot.shape

(723, 62)

In [27]:
clustering_44_grouped = clustering_44_onehot.groupby('Code_INSEE').mean().reset_index()
clustering_44_grouped

,Code_INSEE,basket_Competition_1,basket_Competition_10,basket_Competition_2,basket_Competition_3,basket_Competition_32,basket_Competition_4,basket_Competition_5,basket_Competition_7,basket_Competition_9,...,foot_Training_5,foot_Training_6,foot_Training_8,swim_Competition_1,swim_Competition_2,swim_Competition_5,swim_Leisure_1,swim_Leisure_2,swim_Training_1,swim_Training_2
0,44001,0.00,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,44002,0.00,0.0,0.25,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,44003,0.00,0.0,0.00,0.142857,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857
3,44004,0.25,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,44005,0.25,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,44220,0.00,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
211,44221,0.00,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
212,44222,0.00,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
213,44223,0.25,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [28]:
# Let's confirm the new size :
clustering_44_grouped['Code_INSEE']=clustering_44_grouped['Code_INSEE'].apply(str)
clustering_44_grouped.shape


(215, 62)

In [29]:
# Let's print each district along with the top 5 most common equipements
num_top_sp_equip = 5

for hood in clustering_44_grouped['Code_INSEE']:
    print("----"+hood+"----")
    temp = clustering_44_grouped[clustering_44_grouped['Code_INSEE'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_sp_equip))
    print('\n')

----44001----
                  venue  freq
0    foot_Competition_2   0.5
1      basket_Leisure_2   0.5
2  basket_Competition_1   0.0
3       foot_Training_1   0.0
4    foot_Competition_9   0.0


----44002----
                  venue  freq
0  basket_Competition_2  0.25
1    foot_Competition_2  0.25
2        foot_Leisure_2  0.25
3      basket_Leisure_1  0.25
4  basket_Competition_1  0.00


----44003----
                venue  freq
0     swim_Training_2  0.14
1  foot_Competition_4  0.14
2     foot_Training_2  0.14
3    basket_Leisure_1  0.14
4   basket_Training_1  0.14


----44004----
                  venue  freq
0  basket_Competition_1  0.25
1        foot_Leisure_1  0.25
2    foot_Competition_1  0.25
3      basket_Leisure_1  0.25
4       foot_Training_1  0.00


----44005----
                  venue  freq
0  basket_Competition_1  0.25
1    foot_Competition_2  0.25
2        foot_Leisure_1  0.25
3       foot_Training_1  0.25
4        swim_Leisure_1  0.00


----44006----
                  

                  venue  freq
0  basket_Competition_1   0.5
1    foot_Competition_2   0.5
2        foot_Leisure_7   0.0
3    foot_Competition_8   0.0
4    foot_Competition_9   0.0


----44062----
                  venue  freq
0  basket_Competition_1  0.33
1        foot_Leisure_1  0.33
2       foot_Training_2  0.33
3        foot_Leisure_7  0.00
4    foot_Competition_8  0.00


----44063----
                  venue  freq
0  basket_Competition_2  0.33
1    foot_Competition_2  0.33
2       foot_Training_2  0.33
3  basket_Competition_1  0.00
4        foot_Leisure_7  0.00


----44064----
                  venue  freq
0  basket_Competition_1  0.33
1    foot_Competition_2  0.33
2       foot_Training_1  0.33
3        swim_Leisure_2  0.00
4        swim_Leisure_1  0.00


----44065----
                  venue  freq
0        foot_Leisure_1  0.33
1    foot_Competition_2  0.33
2      basket_Leisure_1  0.33
3  basket_Competition_1  0.00
4       foot_Training_1  0.00


----44066----
                  ve



----44125----
                  venue  freq
0       swim_Training_1  0.25
1    foot_Competition_1  0.25
2        foot_Leisure_3  0.25
3      basket_Leisure_5  0.25
4  basket_Competition_1  0.00


----44126----
                  venue  freq
0    foot_Competition_2   0.5
1      basket_Leisure_1   0.5
2  basket_Competition_1   0.0
3       foot_Training_1   0.0
4    foot_Competition_9   0.0


----44127----
                  venue  freq
0  basket_Competition_1  0.25
1        foot_Leisure_1  0.25
2    foot_Competition_2  0.25
3      basket_Leisure_1  0.25
4       foot_Training_1  0.00


----44128----
                  venue  freq
0       foot_Training_1  0.33
1    foot_Competition_4  0.33
2      basket_Leisure_1  0.33
3  basket_Competition_1  0.00
4    foot_Competition_9  0.00


----44129----
                  venue  freq
0     basket_Training_1   0.2
1  basket_Competition_2   0.2
2    foot_Competition_4   0.2
3       foot_Training_5   0.2
4      basket_Leisure_1   0.2


----44130----
    

4    foot_Competition_9   0.0


----44182----
                  venue  freq
0  basket_Competition_2  0.33
1    foot_Competition_1  0.33
2     basket_Training_1  0.33
3  basket_Competition_1  0.00
4        foot_Leisure_7  0.00


----44183----
                  venue  freq
0  basket_Competition_1  0.25
1    foot_Competition_1  0.25
2        foot_Leisure_1  0.25
3       foot_Training_1  0.25
4        swim_Leisure_1  0.00


----44184----
                  venue  freq
0       swim_Training_1  0.12
1     basket_Training_4  0.12
2       foot_Leisure_14  0.12
3    swim_Competition_2  0.12
4  basket_Competition_7  0.12


----44185----
                venue  freq
0     foot_Training_1   0.5
1  foot_Competition_2   0.5
2      foot_Leisure_7   0.0
3  foot_Competition_8   0.0
4  foot_Competition_9   0.0


----44186----
                  venue  freq
0    foot_Competition_2   0.2
1  basket_Competition_2   0.2
2       swim_Training_1   0.2
3        foot_Leisure_1   0.2
4      basket_Leisure_1   0.2




In [30]:
def return_most_common_equip(row, num_top_sp_equip):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_sp_equip]

In [31]:
num_top_equip = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top equip
columns = ['Code_INSEE']
for ind in np.arange(num_top_equip):
    try:
        columns.append('{}{} Most Common Equip'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Equip'.format(ind+1))

# create a new dataframe
neighborhoods_equip_sorted = pd.DataFrame(columns=columns)
neighborhoods_equip_sorted['Code_INSEE'] = clustering_44_grouped['Code_INSEE']

for ind in np.arange(clustering_44_grouped.shape[0]):
    neighborhoods_equip_sorted.iloc[ind, 1:] = return_most_common_equip(clustering_44_grouped.iloc[ind, :], num_top_equip)

neighborhoods_equip_sorted.head()

,Code_INSEE,1st Most Common Equip,2nd Most Common Equip,3rd Most Common Equip,4th Most Common Equip,5th Most Common Equip,6th Most Common Equip,7th Most Common Equip,8th Most Common Equip,9th Most Common Equip,10th Most Common Equip
0,44001,foot_Competition_2,basket_Leisure_2,swim_Training_2,basket_Leisure_6,foot_Competition_3,foot_Competition_11,foot_Competition_10,foot_Competition_1,basket_Training_6,basket_Training_4
1,44002,basket_Competition_2,foot_Competition_2,basket_Leisure_1,foot_Leisure_2,swim_Training_2,basket_Leisure_7,foot_Competition_3,foot_Competition_11,foot_Competition_10,foot_Competition_1
2,44003,swim_Training_2,basket_Leisure_1,foot_Competition_4,basket_Competition_3,basket_Training_1,foot_Training_2,foot_Leisure_1,basket_Leisure_9,basket_Training_2,basket_Training_6
3,44004,basket_Competition_1,foot_Leisure_1,foot_Competition_1,basket_Leisure_1,basket_Leisure_6,foot_Competition_3,foot_Competition_2,foot_Competition_11,foot_Competition_10,basket_Training_6
4,44005,basket_Competition_1,foot_Competition_2,foot_Leisure_1,foot_Training_1,basket_Leisure_53,foot_Competition_11,foot_Competition_10,foot_Competition_1,basket_Training_6,basket_Training_4


In [37]:
# set number of clusters
kclusters = 5

clustering_44_grouped_clustering = clustering_44_grouped.drop('Code_INSEE', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clustering_44_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 2, 0, 1, 1, 1, 0, 1, 2])

In [40]:
# add clustering labels
#one time only : 
neighborhoods_equip_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

clustering_44_merged = myCP

# merge clustering_44_grouped with clustering_44_data to add latitude/longitude for each district
clustering_44_merged['Code_INSEE']=clustering_44_merged['Code_INSEE'].astype(int)
neighborhoods_equip_sorted['Code_INSEE']=neighborhoods_equip_sorted['Code_INSEE'].astype(int)
clustering_44_merged = clustering_44_merged.merge(neighborhoods_equip_sorted.set_index('Code_INSEE'), on='Code_INSEE')

clustering_44_merged.head() # check the last columns!

,Code_INSEE,Nom_commune,Code_postal,latitude,longitude,Cluster Labels,1st Most Common Equip,2nd Most Common Equip,3rd Most Common Equip,4th Most Common Equip,5th Most Common Equip,6th Most Common Equip,7th Most Common Equip,8th Most Common Equip,9th Most Common Equip,10th Most Common Equip
0,44009,BASSE GOULAINE,44115,47.212014,-1.462469,1,basket_Leisure_1,foot_Competition_2,basket_Competition_3,foot_Training_1,foot_Leisure_1,basket_Training_2,swim_Training_2,basket_Leisure_8,basket_Leisure_9,basket_Training_1
1,44143,REZE,44400,47.176234,-1.549664,2,swim_Training_1,foot_Competition_7,basket_Competition_9,foot_Leisure_18,basket_Leisure_8,swim_Training_2,basket_Leisure_7,foot_Competition_2,foot_Competition_11,foot_Competition_10
2,44150,ST AIGNAN GRANDLIEU,44860,47.133107,-1.623171,2,foot_Competition_3,basket_Leisure_1,swim_Training_2,basket_Leisure_6,foot_Competition_2,foot_Competition_11,foot_Competition_10,foot_Competition_1,basket_Training_6,basket_Training_4
3,44171,ST LEGER LES VIGNES,44710,47.142969,-1.714036,4,basket_Competition_1,foot_Leisure_1,basket_Leisure_6,foot_Competition_3,foot_Competition_2,foot_Competition_11,foot_Competition_10,foot_Competition_1,basket_Training_6,basket_Training_4
4,44194,SAUTRON,44880,47.273227,-1.680473,1,foot_Leisure_6,basket_Training_1,foot_Competition_2,foot_Training_1,basket_Leisure_1,basket_Leisure_6,basket_Leisure_7,basket_Leisure_8,basket_Leisure_9,swim_Training_2


In [41]:
# create map
map_clusters = folium.Map(location=[47.2173, -1.5534], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(clustering_44_merged['latitude'], clustering_44_merged['longitude'], clustering_44_merged['Code_INSEE'], clustering_44_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [48]:
# Examine Clusters :
# "basket ready" districts
clustering_44_merged.loc[clustering_44_merged['Cluster Labels'] == 0, clustering_44_merged.columns[[1] + list(range(5, clustering_44_merged.shape[1]))]]

,Nom_commune,Cluster Labels,1st Most Common Equip,2nd Most Common Equip,3rd Most Common Equip,4th Most Common Equip,5th Most Common Equip,6th Most Common Equip,7th Most Common Equip,8th Most Common Equip,9th Most Common Equip,10th Most Common Equip
16,ST JEAN DE BOISEAU,0,basket_Competition_1,foot_Leisure_1,foot_Competition_1,basket_Leisure_6,foot_Competition_3,foot_Competition_2,foot_Competition_11,foot_Competition_10,basket_Training_6,basket_Training_4
17,INDRE,0,basket_Competition_3,foot_Competition_1,basket_Training_1,basket_Leisure_2,foot_Leisure_2,swim_Training_2,basket_Leisure_7,foot_Competition_3,foot_Competition_2,foot_Competition_11


In [49]:
# Examine Clusters :
# rather "leisure only" district
clustering_44_merged.loc[clustering_44_merged['Cluster Labels'] == 1, clustering_44_merged.columns[[1] + list(range(5, clustering_44_merged.shape[1]))]]

,Nom_commune,Cluster Labels,1st Most Common Equip,2nd Most Common Equip,3rd Most Common Equip,4th Most Common Equip,5th Most Common Equip,6th Most Common Equip,7th Most Common Equip,8th Most Common Equip,9th Most Common Equip,10th Most Common Equip
0,BASSE GOULAINE,1,basket_Leisure_1,foot_Competition_2,basket_Competition_3,foot_Training_1,foot_Leisure_1,basket_Training_2,swim_Training_2,basket_Leisure_8,basket_Leisure_9,basket_Training_1
4,SAUTRON,1,foot_Leisure_6,basket_Training_1,foot_Competition_2,foot_Training_1,basket_Leisure_1,basket_Leisure_6,basket_Leisure_7,basket_Leisure_8,basket_Leisure_9,swim_Training_2
11,BRAINS,1,basket_Competition_1,foot_Competition_2,foot_Leisure_1,basket_Leisure_1,basket_Leisure_6,foot_Competition_3,foot_Competition_11,foot_Competition_10,foot_Competition_1,basket_Training_6
14,LA MONTAGNE,1,basket_Competition_2,foot_Competition_2,foot_Leisure_1,basket_Leisure_1,swim_Training_2,basket_Leisure_7,foot_Competition_3,foot_Competition_11,foot_Competition_10,foot_Competition_1
18,MAUVES SUR LOIRE,1,basket_Competition_1,foot_Competition_2,basket_Leisure_1,foot_Training_1,basket_Leisure_6,foot_Competition_3,foot_Competition_11,foot_Competition_10,foot_Competition_1,basket_Training_6
21,BOUAYE,1,foot_Competition_2,basket_Competition_3,foot_Leisure_1,foot_Training_1,swim_Training_2,basket_Leisure_6,foot_Competition_11,foot_Competition_10,foot_Competition_1,basket_Training_6


In [50]:
# Examine Clusters :
# Center : " Highly Equiped " 
clustering_44_merged.loc[clustering_44_merged['Cluster Labels'] == 2, clustering_44_merged.columns[[1] + list(range(5, clustering_44_merged.shape[1]))]]

,Nom_commune,Cluster Labels,1st Most Common Equip,2nd Most Common Equip,3rd Most Common Equip,4th Most Common Equip,5th Most Common Equip,6th Most Common Equip,7th Most Common Equip,8th Most Common Equip,9th Most Common Equip,10th Most Common Equip
1,REZE,2,swim_Training_1,foot_Competition_7,basket_Competition_9,foot_Leisure_18,basket_Leisure_8,swim_Training_2,basket_Leisure_7,foot_Competition_2,foot_Competition_11,foot_Competition_10
2,ST AIGNAN GRANDLIEU,2,foot_Competition_3,basket_Leisure_1,swim_Training_2,basket_Leisure_6,foot_Competition_2,foot_Competition_11,foot_Competition_10,foot_Competition_1,basket_Training_6,basket_Training_4
5,BOUGUENAIS,2,foot_Competition_3,basket_Competition_3,foot_Training_1,basket_Leisure_2,foot_Leisure_1,swim_Training_1,foot_Competition_11,foot_Competition_10,foot_Competition_1,basket_Leisure_7
6,COUERON,2,foot_Competition_5,basket_Leisure_3,basket_Competition_3,foot_Training_2,basket_Training_4,basket_Leisure_8,basket_Leisure_9,basket_Training_1,basket_Training_2,basket_Training_6
7,NANTES,2,basket_Training_6,basket_Competition_32,basket_Leisure_53,foot_Training_13,foot_Leisure_35,swim_Competition_5,swim_Training_1,foot_Competition_46,foot_Competition_11,foot_Competition_10
8,LE PELLERIN,2,foot_Competition_3,swim_Training_2,foot_Competition_46,foot_Competition_2,foot_Competition_11,foot_Competition_10,foot_Competition_1,basket_Training_6,basket_Training_4,basket_Training_2
9,LES SORINIERES,2,basket_Competition_1,foot_Competition_3,foot_Leisure_1,basket_Training_1,basket_Leisure_6,foot_Competition_2,foot_Competition_11,foot_Competition_10,foot_Competition_1,basket_Training_6
10,THOUARE SUR LOIRE,2,basket_Leisure_1,foot_Competition_3,basket_Competition_2,foot_Training_1,swim_Training_2,basket_Leisure_8,basket_Leisure_9,basket_Training_1,basket_Training_2,basket_Training_6
12,CARQUEFOU,2,basket_Competition_2,foot_Competition_6,basket_Leisure_3,swim_Competition_1,foot_Training_6,basket_Training_2,swim_Training_2,basket_Training_4,basket_Leisure_8,basket_Leisure_9
13,LA CHAPELLE SUR ERDRE,2,foot_Leisure_3,basket_Leisure_1,basket_Competition_3,foot_Competition_8,foot_Training_3,basket_Leisure_8,basket_Leisure_9,basket_Training_1,basket_Training_2,swim_Training_2


In [51]:
# Examine Clusters :
# 
clustering_44_merged.loc[clustering_44_merged['Cluster Labels'] == 4, clustering_44_merged.columns[[1] + list(range(5, clustering_44_merged.shape[1]))]]

,Nom_commune,Cluster Labels,1st Most Common Equip,2nd Most Common Equip,3rd Most Common Equip,4th Most Common Equip,5th Most Common Equip,6th Most Common Equip,7th Most Common Equip,8th Most Common Equip,9th Most Common Equip,10th Most Common Equip
3,ST LEGER LES VIGNES,4,basket_Competition_1,foot_Leisure_1,basket_Leisure_6,foot_Competition_3,foot_Competition_2,foot_Competition_11,foot_Competition_10,foot_Competition_1,basket_Training_6,basket_Training_4


,Nom_commune,Cluster Labels,1st Most Common Equip,2nd Most Common Equip,3rd Most Common Equip,4th Most Common Equip,5th Most Common Equip,6th Most Common Equip,7th Most Common Equip,8th Most Common Equip,9th Most Common Equip,10th Most Common Equip


In [2]:
# add new datas about populations by districts
myDEMOG=pd.read_csv('communes44_CL1.csv',delimiter=';')
# myDEMOG.shape
# (15, 3)
myDEMOG

,Code_INSEE,Nom_commune,Population_totale
0,44020,BOUGUENAIS,19962
1,44026,CARQUEFOU,20494
2,44047,COUERON,22137
3,44035,LA CHAPELLE SUR ERDRE,19933
4,44120,LE PELLERIN,5347
5,44198,LES SORINIERES,8825
6,44109,NANTES,319284
7,44114,ORVAULT,27623
8,44143,REZE,43153
9,44150,ST AIGNAN GRANDLIEU,4019


In [33]:
my_stat_level_foot2B=my_stat_level_foot2.merge(myDEMOG, left_on='Code_INSEE', right_on='Code_INSEE')
my_stat_level_foot2B['nb_equip_by_1000hab']=1000*my_stat_level_foot2B['Nombre_equipements']/my_stat_level_foot2B['Population_totale']
my_stat_level_foot2C=my_stat_level_foot2B.sort_values(by = 'nb_equip_by_1000hab', ascending = False)
df_2D_mask=my_stat_level_foot2C['nb_equip_by_1000hab']>=0.292
my_stat_level_foot2D=my_stat_level_foot2C[df_2D_mask]
my_stat_level_foot2D

,Code_INSEE,Niveau_activite,Nombre_equipements,Nom_commune,Population_totale,nb_equip_by_1000hab
22,44114,Leisure,15,ORVAULT,27623,0.543026
29,44150,Competition,2,ST AIGNAN GRANDLIEU,4019,0.497636
28,44143,Leisure,18,REZE,43153,0.417120
23,44120,Competition,2,LE PELLERIN,5347,0.374042
34,44162,Leisure,16,ST HERBLAIN,47062,0.339977
6,44026,Training,6,CARQUEFOU,20494,0.292769


In [37]:
my_stat_level_basket2B=my_stat_level_basket2.merge(myDEMOG, left_on='Code_INSEE', right_on='Code_INSEE')
my_stat_level_basket2B['nb_equip_by_1000hab']=1000*my_stat_level_basket2B['Nombre_equipements']/my_stat_level_basket2B['Population_totale']
my_stat_level_basket2C=my_stat_level_basket2B.sort_values(by = 'nb_equip_by_1000hab', ascending = False)
df_2D_mask=my_stat_level_basket2C['nb_equip_by_1000hab']>=0.1133
my_stat_level_basket2D=my_stat_level_basket2C[df_2D_mask]
my_stat_level_basket2D

,Code_INSEE,Niveau_activite,Nombre_equipements,Nom_commune,Population_totale,nb_equip_by_1000hab
20,44114,Leisure,9,ORVAULT,27623,0.325815
30,44172,Competition,4,STE LUCE SUR LOIRE,15510,0.257898
25,44150,Leisure,1,ST AIGNAN GRANDLIEU,4019,0.248818
35,44190,Leisure,6,ST SEBASTIEN SUR LOIRE,28063,0.213805
24,44143,Leisure,8,REZE,43153,0.185387
29,44162,Leisure,8,ST HERBLAIN,47062,0.169989
16,44109,Leisure,53,NANTES,319284,0.165996
5,44026,Leisure,3,CARQUEFOU,20494,0.146384
23,44143,Competition,6,REZE,43153,0.139040
11,44047,Leisure,3,COUERON,22137,0.135520


In [38]:
my_stat_level_swim2B=my_stat_level_swim2.merge(myDEMOG, left_on='Code_INSEE', right_on='Code_INSEE')
my_stat_level_swim2B['nb_equip_by_10000hab']=1000*my_stat_level_swim2B['Nombre_equipements']/my_stat_level_swim2B['Population_totale']
my_stat_level_swim2C=my_stat_level_swim2B.sort_values(by = 'nb_equip_by_10000hab', ascending = False)
df_2D_mask=my_stat_level_swim2C['nb_equip_by_10000hab']>=0.0000001
my_stat_level_swim2D=my_stat_level_swim2C[df_2D_mask]
my_stat_level_swim2D

,Code_INSEE,Niveau_activite,Nombre_equipements,Nom_commune,Population_totale,nb_equip_by_10000hab
0,44020,Training,1,BOUGUENAIS,19962,0.050095
1,44026,Competition,1,CARQUEFOU,20494,0.048795
8,44215,Training,1,VERTOU,25633,0.039012
5,44114,Competition,1,ORVAULT,27623,0.036202
6,44143,Training,1,REZE,43153,0.023173
7,44162,Training,1,ST HERBLAIN,47062,0.021249
3,44109,Competition,3,NANTES,319284,0.009396
2,44109,Competition,2,NANTES,319284,0.006264
4,44109,Training,1,NANTES,319284,0.003132
